In [1]:
import pandas as pd
import numpy as np

In [2]:
def answer_one():
    energy = pd.read_excel('Energy Indicators.xls', skiprows=17)
    GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_76261.csv', skiprows=3)
    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx', skiprows=0)

    del energy['Unnamed: 0']
    del energy['Unnamed: 1']
    del GDP['Unnamed: 68']

    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    GDP.rename(columns={'Country Name':'Country'}, inplace=True)

    energy = energy.drop(range(226, 265,1))

    energy.loc[energy['Country'].str.contains('Democratic People\'s Republic of Korea'), 'Country'] = 'North Korea'
    energy.loc[energy['Country'].str.contains('Republic of Korea'), 'Country'] = 'South Korea'
    energy.loc[energy['Country'].str.contains('United States of America'), 'Country'] = 'United States'
    energy.loc[energy['Country'].str.contains('United Kingdom of Great Britain and Northern Ireland'), 'Country'] = 'United Kingdom'
    energy.loc[energy['Country'].str.contains('China, Hong Kong Special Administrative Region'), 'Country'] = 'Hong Kong'
    energy.loc[energy['Country'].str.contains("Bolivia"), 'Country'] = 'Bolivia'
    energy.loc[energy['Country'].str.contains('China, Macao'), 'Country'] = 'Macao'
    energy.loc[energy['Country'].str.contains('Congo, Dem.'), 'Country'] = 'Democratic Republic Congo'
    energy.loc[energy['Country'].str.contains('Congo, Rep.'), 'Country'] = 'Congo'
    energy.loc[energy['Country'].str.contains('Iran'), 'Country'] = 'Iran'
    energy.loc[energy['Country'].str.contains('Venezuela'), 'Country'] = 'Venezuela'
    energy.loc[energy['Country'].str.contains('Macedonia'), 'Country'] = 'Macedonia'
    energy.loc[energy['Country'].str.contains('Tanzania'), 'Country'] = 'Tanzania'
    energy.loc[energy['Country'].str.contains('Palestine'), 'Country'] = 'Palestine'
    energy.loc[energy['Country'].str.contains('Sint Maarten'), 'Country'] = 'Sint Maarten'
    energy.loc[energy['Country'].str.contains('Moldova'), 'Country'] = 'Moldova'
    energy.loc[energy['Country'].str.contains('Micronesia'), 'Country'] = 'Micronesia'
    energy.loc[energy['Country'].str.contains('Falkland Islands'), 'Country'] = 'Falkland Islands'

    energy.loc[~energy['Energy Supply'].astype(str).str.isdigit(), 'Energy Supply'] = np.nan
    energy.loc[~energy['Energy Supply per Capita'].astype(str).str.isdigit(), 'Energy Supply per Capita'] = np.nan
    energy['Country'] = energy['Country'].str.replace('\d+', '', regex=True)

    energy['Energy Supply'] = energy['Energy Supply'] * 1000000

    GDP.loc[GDP['Country'].str.contains('Korea, Rep'), 'Country'] = 'South Korea'
    GDP.loc[GDP['Country'].str.contains('Iran'), 'Country'] = 'Iran'
    GDP.loc[GDP['Country'].str.contains('Hong Kong'), 'Country'] = 'Hong Kong'
    GDP.loc[GDP['Country'].str.contains('Congo, Dem.'), 'Country'] = 'Democratic Republic Congo'
    GDP.loc[GDP['Country'].str.contains('Congo, Rep.'), 'Country'] = 'Congo'
    GDP.loc[GDP['Country'].str.contains('Venezuela'), 'Country'] = 'Venezuela'
    GDP.loc[GDP['Country'].str.contains('Macedonia'), 'Country'] = 'Macedonia'
    GDP.loc[GDP['Country'].str.contains('Micronesia'), 'Country'] = 'Micronesia'
    ScimEn.loc[ScimEn['Country'].str.contains('Micronesia'), 'Country'] = 'Micronesia'
    
    intersection = pd.merge(GDP, pd.merge(energy, ScimEn, on='Country', how='inner'), on='Country', how='inner')
    intersection = intersection[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    intersection = intersection[intersection['Rank'] <= 15].sort_values(['Rank']).reset_index()
    del intersection['index']
    return intersection

In [3]:
def answer_two():
    Top15 = answer_one()
    avgGDP = pd.Series(Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1),name="avgGDP")
    return avgGDP

In [4]:
def answer_three():
    Top15 = answer_one()
    
    TopGDP = answer_two().sort_values(ascending=False)
    TopGDP = TopGDP.reset_index()
    row = Top15.loc[int(TopGDP.loc[5]['index'])]
    
    return row['2015'] - row['2006']

In [5]:
def answer_four():
    Top15 = answer_one()
    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx', skiprows=0)
    Top15['ratio'] = Top15['Self-citations']/Top15['Citations']
    
    maxRatio = max(Top15['ratio'])
    
    rowMax = Top15[Top15['ratio'] == maxRatio]
    
    country = ScimEn.loc[ScimEn['Rank']==max(rowMax['Rank']), 'Country']
    
    return [max(country), maxRatio]

In [6]:
def answer_five():
    Top15 = answer_one()
    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx', skiprows=0)
    
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    row = Top15.sort_values('Population', ascending=False).reset_index().loc[2]
    
    rank = row['Rank']
    
    return max(ScimEn.loc[ScimEn['Rank'] == rank, 'Country'])

In [7]:
def answer_six():
    Top15 = answer_one()
    
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Citable documents per capita'] = Top15["Citable documents"]/Top15['Population']
    
    Top15['Energy Supply per Capita'] = Top15['Energy Supply per Capita'].astype(float)
    Top15['Citable documents per capita'] = Top15['Citable documents per capita'].astype(float)
    
    return Top15['Energy Supply per Capita'].corr(Top15['Citable documents per capita'], method='pearson')

In [8]:
def answer_seven():
    Top15 = answer_one()
    ScimEn = pd.read_excel('scimagojr country rank 1996-2023.xlsx', skiprows=0)
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15['Population'] = Top15['Energy Supply'].astype(float)/Top15['Energy Supply per Capita'].astype(float)
    
    Top15 = pd.merge(Top15, ScimEn[['Country', 'Rank']], on='Rank')
    Top15['Continent'] = Top15['Country'].map(ContinentDict)
    
    answer = Top15.groupby('Continent')['Population'].agg(['count', 'sum', 'mean', 'std'])
    answer.rename(columns={'count': 'size'}, inplace=True)
    
    return answer

In [9]:
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1,402188,400158,5077232,3511673,12.62,343,127191000000,93,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551546e+12,8.532185e+12,9.570471e+12,1.047562e+13,1.106157e+13
1,2,208042,203356,3540288,1004877,17.02,458,90838000000,286,11.570980,1.381558e+13,1.447423e+13,1.476986e+13,1.447807e+13,1.504897e+13,1.559973e+13,1.625397e+13,1.688068e+13,1.760814e+13,1.829502e+13
2,3,81853,79757,968523,369256,11.83,237,33195000000,26,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856722e+12,2.039126e+12,2.103588e+12
3,4,58342,57735,717946,154848,12.31,228,18984000000,149,10.232820,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
4,5,56288,54713,1095371,179764,19.46,293,7920000000,124,10.600470,2.708442e+12,3.090510e+12,2.929412e+12,2.412840e+12,2.485483e+12,2.663806e+12,2.707090e+12,2.784854e+12,3.064708e+12,2.927911e+12
5,6,50906,49773,777362,160302,15.27,252,13261000000,165,17.901530,3.046309e+12,3.484057e+12,3.808786e+12,3.479801e+12,3.468154e+12,3.824829e+12,3.597897e+12,3.808086e+12,3.965801e+12,3.423568e+12
6,7,46186,45868,217996,96688,4.72,112,30709000000,214,17.288680,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
7,8,41209,40390,915491,142691,22.22,284,10431000000,296,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
8,9,38700,36909,639473,147302,16.52,209,6530000000,109,33.667230,1.958564e+12,2.222524e+12,2.417508e+12,2.209484e+12,2.144936e+12,2.306974e+12,2.097929e+12,2.153226e+12,2.173256e+12,1.845428e+12
9,10,38362,38046,618762,105419,16.13,198,11007000000,221,2.279353,1.053217e+12,1.172614e+12,1.047339e+12,9.439419e+11,1.143672e+12,1.253290e+12,1.278047e+12,1.370633e+12,1.484489e+12,1.466039e+12


In [10]:
answer_two()

0     6.927707e+12
1     1.572243e+13
2     1.602352e+12
3     5.239642e+12
4     2.777505e+12
5     3.590729e+12
6     1.666746e+12
7     1.616359e+12
8     2.152983e+12
9     1.221328e+12
10    4.567516e+11
11    2.692000e+12
12    1.406644e+12
13    1.207997e+12
14    1.988889e+12
Name: avgGDP, dtype: float64

In [11]:
answer_three()

124621907951.68018

In [12]:
answer_four()

['China', 0.6916510807463594]

In [13]:
answer_five()

'United States'

In [14]:
answer_six()

0.7019601723071185

In [15]:
answer_seven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
